In [7]:
import pandas as pd
datastatics = {}
thred_score = {"en": 20, "eu": 20, "jp": 0}
for dataset_name in ['eu', 'en', 'jp']:
    datastatics[dataset_name] = {}
    df = pd.read_excel(f'results/{dataset_name}/{dataset_name}_results.xlsx')
    final_df = pd.read_excel(f'results/{dataset_name}-final.xlsx')
    sub_df = df[~df['job_description'].isin(final_df['job_description'])]
    sub_df = pd.concat([sub_df, final_df])
    datastatics[dataset_name]['origin_length'] = len(sub_df)
    # sub_df.to_excel(f'results/{dataset_name}-llm.xlsx', index=None)
    sub_df = sub_df[sub_df['score']>=thred_score[dataset_name]]
    sub_df_group = sub_df.groupby('task').agg(list)[['job_description']]
    sub_df_group['len'] = sub_df_group['job_description'].apply(len)
    sub_df_group = sub_df_group.sort_values('len', ascending=False)
    for thred_len in [0, 5, 10, 15, 20, 25, 40]:
        all_df = sub_df_group[sub_df_group['len']>=thred_len].explode('job_description')[['job_description']].reset_index()
        if thred_len == 0:
            all_df.to_excel(f'results/{dataset_name}/{dataset_name}_all.xlsx', index=None)
            datastatics[dataset_name]['length'] = len(all_df)
        datastatics[dataset_name][thred_len] = all_df['task'].nunique()
pd.DataFrame(datastatics)

,eu,en,jp
origin_length,6344,7752,2736
length,5812,7177,2628
0,1107,1510,873
5,404,533,157
10,162,185,52
15,73,81,13
20,43,39,7
25,23,20,4
40,6,1,1
